In [69]:
import sys
import os
sys.path.append("/content/ml-project-2-pytyeee")

REPO_DIR = "ml-project-2-pytyeee/"
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/ml-project-2-pytyeee/', '/content/ml-project-2-pytyeee/', '/content/ml-project-2-pytyeee/', '/content/ml-project-2-pytyeee/', '/content/ml-project-2-pytyeee/', '/content/ml-project-2-pytyeee', '/content/ml-project-2-pytyeee']


In [70]:
import segmentation_models_pytorch as smp
from scripts.preprocessing import *
from torch.utils.data import DataLoader, TensorDataset
import torch

In [80]:
NUM_CPU = 2 # os.cpu_count()
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

TRAINING_SIZE = 100
BATCH_SIZE = 32
NUM_EPOCHS = 10
BASE_LR = 0.001

ENCODER = 'resnet34'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid'

In [72]:
train_dir = REPO_DIR + "dataset/training/"
train_images_filename = train_dir + "images/"
train_masks_filename = train_dir + "groundtruth/"

train_images = extract_data(train_images_filename, TRAINING_SIZE)
train_masks = extract_data(train_masks_filename, TRAINING_SIZE)
print(f"Input shapes: {train_images.shape, train_masks.shape}")

Loading ml-project-2-pytyeee/dataset/training/images/satImage_001.png
Loading ml-project-2-pytyeee/dataset/training/images/satImage_100.png
Loading ml-project-2-pytyeee/dataset/training/groundtruth/satImage_001.png
Loading ml-project-2-pytyeee/dataset/training/groundtruth/satImage_100.png
Input shapes: ((100, 400, 400, 3), (100, 400, 400))


In [73]:
# Resize images to a size divisible by 32 to make it UNet compatible
train_data = img_resize(np.transpose(train_images, (0, 3, 1, 2)), (416, 416))
train_labels = img_resize(np.transpose(np.expand_dims(train_masks, -1), (0, 3, 1, 2)),  (416, 416))

# 1 hot encode each non black pixel to 1 pixel
train_labels[train_labels > 0] = 1
print(f"New shapes: {train_data.shape, train_labels.shape}")

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


New shapes: (torch.Size([100, 3, 416, 416]), torch.Size([100, 1, 416, 416]))


In [74]:
# Create segmentation model with pretrained encoder
model = smp.Unet(
    encoder_name=ENCODER,
    encoder_weights=ENCODER_WEIGHTS,
    classes=1, # 1 for grayscale
    activation=ACTIVATION,
)

model = model.to(DEVICE)
# preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [81]:
# Create dataset
road_dataset = TensorDataset(train_data, train_labels)

# Get train and val data loaders
train_loader = DataLoader(road_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_CPU)

In [76]:
# define loss
criterion = smp.losses.DiceLoss('binary')

# define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=BASE_LR)

# define learning rate scheduler (not used in this NB)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer, T_0=1, T_mult=2, eta_min=5e-5,
)

In [77]:
def train_epoch(model, optimizer, scheduler, criterion, train_loader, epoch, device):
    # ***************************************************
    # Set model to training mode (affects dropout, batch norm e.g.)
    model.train()

    loss_history = []
    accuracy_history = []
    f1_history = []
    lr_history = []

    for batch_idx, (data, target) in enumerate(train_loader):
        # Move the data to the device
        data, target = data.to(device), target.to(device)
        # Zero the gradients
        optimizer.zero_grad()
        # Compute model output
        output = model(data)
        output = torch.where(output > 0.5, 1., 0.).requires_grad_()
        # Compute loss
        loss = criterion(output, target)
        # Backpropagate loss
        loss.backward()
        # Perform an optimizer step
        optimizer.step()
        # Perform a learning rate scheduler step
        scheduler.step()
        # Compute accuracy_float (float value, not a tensor)
        accuracy_float = torch.sum((output==target)) / torch.numel(target)
        # Compute loss_float (float value, not a tensor)
        loss_float = loss.item()
        # Add loss_float to loss_history
        loss_history.append(loss_float)
        # Add accuracy_float to accuracy_history
        accuracy_history.append(accuracy_float)
        # # Add learning rate to lr_history
        # lr_history.append(scheduler.get_last_lr()[0])

        print(
            f"Train Epoch: {epoch}-{batch_idx:03d} "
            f"batch_loss={loss_float:0.2e} "
            f"batch_acc={accuracy_float:0.3f} "
            # f"lr={scheduler.get_last_lr()[0]:0.3e} "
        )

    return loss_history, accuracy_history, lr_history, output, target

In [82]:
lr_history = []
train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []
for epoch in range(1, NUM_EPOCHS + 1):
    train_loss, train_acc, lrs, output, target = train_epoch(
        model, optimizer, lr_scheduler, criterion, train_loader, epoch, DEVICE
    )
    train_loss_history.extend(train_loss)
    train_acc_history.extend(train_acc)
    lr_history.extend(lrs)

Train Epoch: 1-000 batch_loss=6.41e-01 batch_acc=0.442 
Train Epoch: 1-001 batch_loss=6.23e-01 batch_acc=0.447 
Train Epoch: 1-002 batch_loss=6.55e-01 batch_acc=0.437 
Train Epoch: 1-003 batch_loss=6.33e-01 batch_acc=0.439 
Train Epoch: 2-000 batch_loss=6.57e-01 batch_acc=0.437 
Train Epoch: 2-001 batch_loss=6.42e-01 batch_acc=0.437 
Train Epoch: 2-002 batch_loss=6.20e-01 batch_acc=0.450 
Train Epoch: 2-003 batch_loss=6.42e-01 batch_acc=0.443 
Train Epoch: 3-000 batch_loss=6.30e-01 batch_acc=0.445 
Train Epoch: 3-001 batch_loss=6.36e-01 batch_acc=0.441 
Train Epoch: 3-002 batch_loss=6.58e-01 batch_acc=0.429 
Train Epoch: 3-003 batch_loss=6.14e-01 batch_acc=0.443 
Train Epoch: 4-000 batch_loss=6.34e-01 batch_acc=0.444 
Train Epoch: 4-001 batch_loss=6.48e-01 batch_acc=0.436 
Train Epoch: 4-002 batch_loss=6.31e-01 batch_acc=0.445 
Train Epoch: 4-003 batch_loss=6.88e-01 batch_acc=0.416 
Train Epoch: 5-000 batch_loss=6.27e-01 batch_acc=0.446 
Train Epoch: 5-001 batch_loss=6.46e-01 batch_acc